# MCP Jupyter Setup for Claude Code

This notebook helps you set up MCP (Model Context Protocol) integration between Claude Code and Jupyter notebooks.

## Architecture
```
Neovim/VSCode → Claude Code CLI → MCP Jupyter Server → Jupyter Kernel
```

## Contents
1. Check Prerequisites
2. Install MCP Jupyter Server
3. Register with Claude Code
4. Verify Setup
5. Test MCP Tools

---
## 1. Check Prerequisites

In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")

# Check if Python >= 3.8
assert sys.version_info >= (3, 8), "Python 3.8+ required"
print("✓ Python version OK")

In [ ]:
# Check Jupyter installation
import subprocess

def check_command(cmd, name):
    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        print(f"✓ {name} installed")
        return True
    except FileNotFoundError:
        print(f"✗ {name} not found")
        return False

check_command(["jupyter", "--version"], "Jupyter")
check_command(["jupyter", "lab", "--version"], "JupyterLab")

In [ ]:
# Check Claude Code CLI
import shutil

claude_path = shutil.which("claude")
if claude_path:
    print(f"✓ Claude Code CLI found: {claude_path}")
    result = subprocess.run(["claude", "--version"], capture_output=True, text=True)
    print(f"  Version: {result.stdout.strip()}")
else:
    print("✗ Claude Code CLI not found")
    print("  Install with: curl -fsSL https://claude.ai/install.sh | bash")

---
## 2. Install MCP Jupyter Server

Choose one of the two options:

### Option A: ClaudeJupy (Simple, Recommended)
- Persistent Python kernel
- Basic notebook cell manipulation

### Option B: Datalayer Jupyter MCP (Full Control)
- Full JupyterLab integration
- Complete notebook manipulation

In [ ]:
# Option A: Install ClaudeJupy
# Uncomment to install:

# !pip install ml-jupyter-mcp

# Or with pipx (isolated environment):
# !pipx install ml-jupyter-mcp

print("To install ClaudeJupy, run:")
print("  pip install ml-jupyter-mcp")
print("  or")
print("  pipx install ml-jupyter-mcp")

In [ ]:
# Option B: Install Datalayer Jupyter MCP
# Uncomment to install:

# !pip install jupyterlab==4.4.1 jupyter-collaboration==4.0.2 jupyter-mcp-tools>=0.1.4 ipykernel
# !pip uninstall -y pycrdt datalayer_pycrdt
# !pip install datalayer_pycrdt==0.12.17

print("To install Datalayer Jupyter MCP, run:")
print("  pip install jupyterlab==4.4.1 jupyter-collaboration==4.0.2 jupyter-mcp-tools>=0.1.4 ipykernel")
print("  pip uninstall -y pycrdt datalayer_pycrdt")
print("  pip install datalayer_pycrdt==0.12.17")

---
## 3. Register MCP Server with Claude Code

In [ ]:
# Option A: Register ClaudeJupy
# Run this in terminal:

register_claudejupy_cmd = "claude mcp add jupyter-executor ml-jupyter-mcp"

print("To register ClaudeJupy with Claude Code, run in terminal:")
print(f"  {register_claudejupy_cmd}")
print("")

# Uncomment to run directly:
# !claude mcp add jupyter-executor ml-jupyter-mcp

In [ ]:
# Option B: Register Datalayer Jupyter MCP
import json

jupyter_mcp_config = {
    "command": "uvx",
    "args": ["jupyter-mcp-server@latest"],
    "env": {
        "JUPYTER_URL": "http://localhost:8888",
        "JUPYTER_TOKEN": "MY_TOKEN",  # Change this!
        "ALLOW_IMG_OUTPUT": "true"
    }
}

config_json = json.dumps(jupyter_mcp_config)

print("To register Datalayer Jupyter MCP, run in terminal:")
print(f'  claude mcp add-json "jupyter" \'{config_json}\'')
print("")
print("Don't forget to:")
print("  1. Change MY_TOKEN to your actual Jupyter token")
print("  2. Start JupyterLab with: jupyter lab --port 8888 --IdentityProvider.token MY_TOKEN")

---
## 4. Verify Setup

In [ ]:
# Check registered MCP servers
print("Checking registered MCP servers...")
print("Run in terminal: claude mcp list")
print("")

# Try to run it
try:
    result = subprocess.run(["claude", "mcp", "list"], capture_output=True, text=True, timeout=10)
    print("Output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
except subprocess.TimeoutExpired:
    print("Command timed out - run manually in terminal")
except FileNotFoundError:
    print("Claude CLI not found in PATH")

In [ ]:
# Check MCP config file location
import os
from pathlib import Path

# Possible config locations
config_paths = [
    Path.home() / ".config" / "claude-code" / "mcp.json",
    Path.home() / ".claude" / "mcp.json",
    Path.home() / "AppData" / "Roaming" / "claude-code" / "mcp.json",  # Windows
]

print("Checking for MCP config files:")
for path in config_paths:
    if path.exists():
        print(f"✓ Found: {path}")
        try:
            with open(path) as f:
                config = json.load(f)
            print(f"  Servers: {list(config.get('servers', {}).keys())}")
        except Exception as e:
            print(f"  Error reading: {e}")
    else:
        print(f"✗ Not found: {path}")

---
## 5. Test MCP Tools

Once registered, you can use these tools in Claude Code:

In [ ]:
# Example MCP tool calls for ClaudeJupy
print("=" * 60)
print("ClaudeJupy Tool Examples")
print("=" * 60)
print("""
In Claude Code, you can say:

1. Execute Python code:
   "Use jupyter-executor to run: x = 42; print(x)"

2. Add a cell to a notebook:
   "Use jupyter-executor to add a code cell to analysis.ipynb that imports pandas"

3. Run with persistent state:
   "Use jupyter-executor to define a function, then call it in the next command"

The actual MCP tool calls look like:
  mcp__jupyter-executor__execute_code("x = 42")
  mcp__jupyter-executor__add_notebook_cell("analysis.ipynb", "code", "import pandas as pd")
""")

In [ ]:
# Example MCP tool calls for Datalayer Jupyter
print("=" * 60)
print("Datalayer Jupyter MCP Tool Examples")
print("=" * 60)
print("""
In Claude Code, you can say:

1. Connect to a notebook:
   "Use jupyter to open notebooks/analysis.ipynb"

2. Insert a new cell:
   "Use jupyter to insert a code cell at the end with: import matplotlib.pyplot as plt"

3. Execute a cell:
   "Use jupyter to execute cell 3 in the notebook"

4. Run all cells:
   "Use jupyter to run all cells in the notebook"

Available tools:
  - use_notebook
  - insert_cell  
  - execute_cell
  - notebook_run-all-cells
  - list_notebooks
""")

---
## 6. Neovim Integration (Optional)

If using Neovim with claudecode.nvim:

In [ ]:
# claudecode.nvim configuration example
neovim_config = '''
-- lazy.nvim plugin spec for claudecode.nvim
return {
  {
    "coder/claudecode.nvim",
    version = "*",
    dependencies = { "folke/snacks.nvim" },
    opts = {
      terminal_cmd = "claude",
      
      -- Terminal configuration
      terminal = {
        -- Options: "snacks", "native", "external", "none"
        provider = "snacks",
        split_side = "right",
        split_width_percentage = 0.30,
        
        -- For external terminal (Alacritty/Kitty):
        -- provider = "external",
        -- provider_opts = {
        --   external_terminal_cmd = "alacritty -e %s",
        -- },
      },
      
      git_repo_cwd = true,
      auto_start = true,
    },
    keys = {
      { "<C-,>", "<cmd>ClaudeCodeFocus<cr>", desc = "Claude Code", mode = { "n", "x" } },
    },
  },
}
'''

print("Neovim claudecode.nvim Configuration:")
print(neovim_config)

In [ ]:
# Save neovim config to file
config_dir = Path("../config")
config_dir.mkdir(exist_ok=True)

nvim_config_path = config_dir / "claudecode_nvim_example.lua"
with open(nvim_config_path, "w") as f:
    f.write(neovim_config)

print(f"Saved example Neovim config to: {nvim_config_path}")

---
## 7. Quick Setup Script

In [ ]:
# Create a quick setup script
setup_script = '''#!/bin/bash
# MCP Jupyter Quick Setup for Claude Code

echo "========================================"
echo "MCP Jupyter Setup for Claude Code"
echo "========================================"

# Check Claude Code CLI
if ! command -v claude &> /dev/null; then
    echo "Error: Claude Code CLI not found"
    echo "Install with: curl -fsSL https://claude.ai/install.sh | bash"
    exit 1
fi

echo "Claude Code CLI found: $(claude --version)"

# Install ClaudeJupy
echo ""
echo "Installing ClaudeJupy MCP server..."
pip install ml-jupyter-mcp

# Register with Claude Code
echo ""
echo "Registering MCP server with Claude Code..."
claude mcp add jupyter-executor ml-jupyter-mcp

# Verify
echo ""
echo "Verifying setup..."
claude mcp list

echo ""
echo "========================================"
echo "Setup complete!"
echo "========================================"
echo ""
echo "You can now use jupyter-executor in Claude Code."
echo "Example: 'Use jupyter-executor to run print(42)'"
'''

scripts_dir = Path("../scripts")
scripts_dir.mkdir(exist_ok=True)

script_path = scripts_dir / "setup_mcp_jupyter.sh"
with open(script_path, "w") as f:
    f.write(setup_script)

# Make executable
import stat
script_path.chmod(script_path.stat().st_mode | stat.S_IEXEC)

print(f"Created setup script: {script_path}")
print("")
print("Run with: bash scripts/setup_mcp_jupyter.sh")

---
## Summary

### Quick Start (ClaudeJupy)
```bash
# Install
pip install ml-jupyter-mcp

# Register
claude mcp add jupyter-executor ml-jupyter-mcp

# Verify
claude mcp list
```

### Quick Start (Datalayer Jupyter)
```bash
# Install dependencies
pip install jupyterlab jupyter-mcp-tools ipykernel

# Start JupyterLab
jupyter lab --port 8888 --IdentityProvider.token YOUR_TOKEN

# Register (in another terminal)
claude mcp add-json "jupyter" '{"command":"uvx","args":["jupyter-mcp-server@latest"],"env":{"JUPYTER_URL":"http://localhost:8888","JUPYTER_TOKEN":"YOUR_TOKEN"}}'

# Verify
claude mcp list
```

### Troubleshooting
1. `claude --version` should work
2. `:ClaudeCodeStatus` in Neovim should show "connected"
3. `claude mcp list` should show your server as Connected

In [ ]:
print("\n" + "=" * 60)
print("MCP Jupyter Setup Notebook Complete!")
print("=" * 60)
print("""
Files created:
  - ../config/claudecode_nvim_example.lua
  - ../scripts/setup_mcp_jupyter.sh

Documentation:
  - ../research/mcp_jupyter_setup_guide.md

Next steps:
  1. Run the setup script or follow manual steps above
  2. Verify with 'claude mcp list'
  3. Test in Claude Code with 'Use jupyter-executor to run print(42)'
""")